In [128]:
import requests as rq
import json
import pandas as pd

In [129]:
dt_page = rq.get('https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html')
assert(dt_page.ok)

In [130]:
dt_content = dt_page.content.decode('utf-8')

In [131]:
pos1 = dt_content.find('>Updated')+9
pos2 = dt_content.find('<', pos1)
today = (pd.to_datetime(dt_content[pos1: pos2])-pd.to_timedelta('1D')).strftime('%Y-%m-%d')

In [132]:
page = rq.get('https://www.cdc.gov/coronavirus/2019-ncov/map-cases-us.json')
assert(page.ok)

In [133]:
content = page.content.decode('utf-8')
j = json.loads(content)

In [134]:
df = pd.DataFrame(j['data'])
df['scrape_dt'] = today

In [137]:
states = pd.read_csv('static-data/states.csv', index_col='state_full')

In [146]:
df2 = df.join(states, how='left', on='Jurisdiction')

In [147]:
df2[df2.state_code.isna()].Jurisdiction

2        American Samoa
12                 Guam
22     Marshall Islands
26           Micronesia
39    Northern Marianas
43                Palau
45          Puerto Rico
53       Virgin Islands
59                 None
Name: Jurisdiction, dtype: object

In [149]:
df_out = df2.loc[~df2.state_code.isna(),['scrape_dt','state_code','Cases Reported']]
df_out.columns = ['dt','state','confirmed']
df_out.head(3)

,dt,state,confirmed
0,2020-03-23,AL,196
1,2020-03-23,AK,36
3,2020-03-23,AZ,253


In [150]:
df_out.to_csv(f'state-data/{today}.csv', index=False, encoding='utf-8')